In [2]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import EarlyStopping

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces in an image and return the cropped faces
def detect_faces(img, target_size=(224, 224)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    face_imgs = []

    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, target_size)
        face_imgs.append(face)
    
    return face_imgs

# Function to preprocess images for VGG19
def preprocess_images(imgs):
    img_arrays = np.array([img.astype('float32') / 255.0 for img in imgs])
    return img_arrays

# Function to process all images in a directory and return detected faces
def process_directory(directory, target_size=(224, 224)):
    all_faces = []
    all_labels = []

    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                faces = detect_faces(img, target_size)
                
                if faces:
                    faces_preprocessed = preprocess_images(faces)
                    all_faces.extend(faces_preprocessed)
                    label = os.path.basename(root)
                    all_labels.extend([label] * len(faces_preprocessed))
    
    return np.array(all_faces), np.array(all_labels)

# Example usage: process train, validation, and test directories
train_dir = 'D:\\ELC_2024\\Accidence Avoidance\\Dataset_seg - Copy\\train'
val_dir = "D:\\ELC_2024\\Accidence Avoidance\\Dataset_seg - Copy\\val"
#test_dir = 'D:\\final\\test'

x_train, y_train = process_directory(train_dir)
x_val, y_val = process_directory(val_dir)
#x_test, y_test = process_directory(test_dir)

# Ensure labels are numeric
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
#y_test = label_encoder.transform(y_test)

# Load VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

# Train the model
model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_val, y_val),
    callbacks=[early_stop],
    batch_size=32
)

# Save the trained model
#model.save("vgg19_facial_expression_model.h5")


Epoch 1/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 502s 889ms/step - accuracy: 0.5783 - loss: 1.8940 - val_accuracy: 0.7338 - val_loss: 0.7120
Epoch 2/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 502s 893ms/step - accuracy: 0.7104 - loss: 0.7474 - val_accuracy: 0.6865 - val_loss: 0.7843
Epoch 3/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 500s 890ms/step - accuracy: 0.7356 - loss: 0.6816 - val_accuracy: 0.7259 - val_loss: 0.7123
Epoch 4/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 499s 888ms/step - accuracy: 0.7471 - loss: 0.6385 - val_accuracy: 0.7477 - val_loss: 0.6568
Epoch 5/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 500s 890ms/step - accuracy: 0.7704 - loss: 0.5876 - val_accuracy: 0.7538 - val_loss: 0.6078
Epoch 6/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 513s 914ms/step - accuracy: 0.7701 - loss: 0.5695 - val_accuracy: 0.7585 - val_loss: 0.6088
Epoch 7/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 503s 895ms/step - accuracy: 0.7837 - loss: 0.5556 - val_accuracy: 0.7589 - val_loss: 0.5940
Epoch 8/20
562/562 ━━━━━━━━━━━━━━━━━━━━ 500s 890ms/step - accuracy: 0.7817 -

In [3]:
model.save("D:\\ELC_2024\\Accidence Avoidance\\VGG19\\VGG19_New.keras")

In [4]:
import cv2
import numpy as np
from keras.models import load_model

# Load the trained VGG16 model
model = load_model("D:\\ELC_2024\\Accidence Avoidance\\VGG19\\VGG19_New.keras")

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define labels (update with your actual labels)
labels = ['closed eyes', 'drowsy', 'half-closed', 'neutral', 'yawning']

# Function to preprocess a single image
def preprocess_single_image(img, target_size=(224, 224)):
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Frame rate (frames per second)
frame_rate = 30

# Calculate the interval between frames in milliseconds
frame_interval = int(1000 / frame_rate)

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture image")
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face from the frame
        face = frame[y:y+h, x:x+w]

        # Preprocess the face
        preprocessed_face = preprocess_single_image(face)
        
        # Predict the label of the face
        predictions = model.predict(preprocessed_face)
        label_index = np.argmax(predictions)
        label = labels[label_index]
        
        # Draw rectangle around the face and put label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f"Prediction: {label}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(frame_interval) & 0xFF == ord('q'):
        break

# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━

In [5]:
%%cmd
python --version

Microsoft Windows [Version 10.0.22631.3880]
(c) Microsoft Corporation. All rights reserved.

(elc_2024) D:\ELC_2024\Accidence Avoidance\VGG19>python --version
Python 3.11.9

(elc_2024) D:\ELC_2024\Accidence Avoidance\VGG19>